# 📊 F1 YouTube Descriptive Analytics - EDA & Visualizations

## Notebook 04: Exploratory Data Analysis & Visualization

This notebook computes and visualizes all requested analytics metrics:

### 1. Driver Engagement Analytics
- Driver Share of Voice (SoV)
- Driver Sentiment Score
- Headline Impact
- Rivalry Intensity

### 2. Audience Sentiment & Reaction
- Global Sentiment Distribution
- Top Fan Favorite Comments
- Controversy Index
- Polarity vs Performance Correlation

### 3. Video Performance Metrics
- Engagement Rate Rankings
- Video Reach Rankings
- Virality Potential

### 4. Content & Topic Analysis
- Dominant Keyword Cloud
- Team Mention Frequency
- Temporal Activity Heatmap

In [ ]:
# Setup and imports
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')

from src import config
from src import utils
from src.analytics import F1DescriptiveAnalytics

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✅ Imports successful!")

In [ ]:
# Load data
videos = pd.read_csv(config.FEATURES_CSV)
comments = pd.read_csv(config.PROCESSED_DATA_DIR / 'comments_with_features.csv')

# Convert string columns back to lists for analysis
comments['drivers_mentioned'] = comments['drivers_mentioned_str'].apply(utils.string_to_list)
comments['teams_mentioned'] = comments['teams_mentioned_str'].apply(utils.string_to_list)
comments['rivalries_detected'] = comments['rivalries_detected_str'].apply(
    lambda x: [tuple(r.split('vs')) for r in x.split('|') if r] if pd.notna(x) and x else []
)

videos['drivers_in_title'] = videos['drivers_in_title_str'].apply(utils.string_to_list)

print(f"📊 Loaded {len(videos)} videos and {len(comments)} comments")

In [ ]:
# Initialize analytics engine
analytics = F1DescriptiveAnalytics(videos, comments)

---
# 🏎️ 1. DRIVER ENGAGEMENT ANALYTICS

## 1.1 Driver Share of Voice (SoV)
*Which drivers are most discussed in comments?*

In [ ]:
# Calculate Driver Share of Voice
driver_sov = analytics.driver_share_of_voice()

# Visualize top 15 drivers by mention count
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart - Mention Count
top_drivers = driver_sov.head(15)
colors = [config.TEAM_COLORS.get(
    config.DRIVERS.get(d, {}).get('team', '').lower().replace(' ', '_'), '#808080'
) for d in top_drivers['driver']]

axes[0].barh(range(len(top_drivers)), top_drivers['mention_count'], color=colors)
axes[0].set_yticks(range(len(top_drivers)))
axes[0].set_yticklabels([config.DRIVERS.get(d, {}).get('full_name', d) for d in top_drivers['driver']])
axes[0].invert_yaxis()
axes[0].set_xlabel('Number of Mentions')
axes[0].set_title('🏎️ Driver Share of Voice - Mention Count')

# Pie chart - Share of Voice
top_10 = driver_sov.head(10)
axes[1].pie(top_10['share_of_voice_pct'], labels=[config.DRIVERS.get(d, {}).get('full_name', d)[:15] for d in top_10['driver']],
            autopct='%1.1f%%', startangle=90)
axes[1].set_title('📊 Top 10 Drivers - Share of Voice %')

plt.tight_layout()
plt.show()

driver_sov.head(10)

## 1.2 Driver Sentiment Score
*Are comments about certain drivers more positive or negative?*

In [ ]:
# Calculate Driver Sentiment Scores
driver_sentiment = analytics.driver_sentiment_scores()

# Filter drivers with enough data
driver_sentiment_filtered = driver_sentiment[driver_sentiment['comment_count'] >= 10].head(15)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Average sentiment bar chart
colors = ['green' if x > 0 else 'red' for x in driver_sentiment_filtered['avg_sentiment']]
axes[0].barh(range(len(driver_sentiment_filtered)), driver_sentiment_filtered['avg_sentiment'], color=colors)
axes[0].set_yticks(range(len(driver_sentiment_filtered)))
axes[0].set_yticklabels(driver_sentiment_filtered['driver'])
axes[0].invert_yaxis()
axes[0].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
axes[0].set_xlabel('Average Sentiment Score (-1 to +1)')
axes[0].set_title('🎭 Driver Sentiment Score')

# Positive vs Negative percentage
x = range(len(driver_sentiment_filtered))
width = 0.35
axes[1].bar([i - width/2 for i in x], driver_sentiment_filtered['positive_pct'], width, label='Positive %', color='green', alpha=0.7)
axes[1].bar([i + width/2 for i in x], driver_sentiment_filtered['negative_pct'], width, label='Negative %', color='red', alpha=0.7)
axes[1].set_xticks(x)
axes[1].set_xticklabels(driver_sentiment_filtered['driver'], rotation=45, ha='right')
axes[1].set_ylabel('Percentage')
axes[1].set_title('📊 Positive vs Negative Comments by Driver')
axes[1].legend()

plt.tight_layout()
plt.show()

driver_sentiment_filtered[['driver', 'avg_sentiment', 'positive_pct', 'negative_pct', 'comment_count']].head(10)

## 1.3 Headline Impact
*Which drivers in video titles generate the most views?*

In [ ]:
# Calculate Headline Impact
headline_impact = analytics.headline_impact()

# Visualize
fig, ax = plt.subplots(figsize=(12, 6))

if not headline_impact.empty:
    top_headline = headline_impact.head(12)
    
    x = range(len(top_headline))
    ax.bar(x, top_headline['avg_views'] / 1e6, color='steelblue', alpha=0.8)
    ax.set_xticks(x)
    ax.set_xticklabels([config.DRIVERS.get(d, {}).get('full_name', d) for d in top_headline['driver']], rotation=45, ha='right')
    ax.set_ylabel('Average Views (Millions)')
    ax.set_title('📺 Headline Impact: Avg Views for Videos Featuring Driver in Title')
    
    # Add video count annotations
    for i, row in enumerate(top_headline.itertuples()):
        ax.annotate(f'n={row.video_count}', (i, row.avg_views/1e6 + 0.1), ha='center', fontsize=8)
else:
    ax.text(0.5, 0.5, 'No driver headlines found', ha='center', va='center', transform=ax.transAxes)

plt.tight_layout()
plt.show()

headline_impact.head(10)

## 1.4 Rivalry Intensity
*Which rivalry pairs generate the most discussion?*

In [ ]:
# Calculate Rivalry Intensity
rivalry = analytics.rivalry_intensity()

fig, ax = plt.subplots(figsize=(12, 5))

if not rivalry.empty and rivalry['comment_count'].sum() > 0:
    rivalry_sorted = rivalry.sort_values('comment_count', ascending=True)
    
    # Create rivalry labels
    labels = [f"{config.DRIVERS.get(row['driver_1'], {}).get('full_name', row['driver_1']).split()[1]} vs {config.DRIVERS.get(row['driver_2'], {}).get('full_name', row['driver_2']).split()[-1]}" 
              for _, row in rivalry_sorted.iterrows()]
    
    ax.barh(range(len(rivalry_sorted)), rivalry_sorted['comment_count'], color='darkorange')
    ax.set_yticks(range(len(rivalry_sorted)))
    ax.set_yticklabels(labels)
    ax.set_xlabel('Number of Co-mentions')
    ax.set_title('⚔️ Rivalry Intensity: Comments Mentioning Both Drivers')
else:
    ax.text(0.5, 0.5, 'No rivalry co-mentions detected', ha='center', va='center', transform=ax.transAxes)

plt.tight_layout()
plt.show()

rivalry

---
# 🎭 2. AUDIENCE SENTIMENT & REACTION

## 2.1 Global Sentiment Distribution
*What's the overall mood of F1 YouTube comments?*

In [ ]:
# Global Sentiment Distribution
sentiment_dist = analytics.global_sentiment_distribution()

fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Pie chart
labels = ['Positive', 'Neutral', 'Negative']
sizes = [sentiment_dist['positive_pct'], sentiment_dist['neutral_pct'], sentiment_dist['negative_pct']]
colors = ['#2ecc71', '#95a5a6', '#e74c3c']
explode = (0.05, 0, 0)

axes[0].pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
axes[0].set_title('🎭 Global Sentiment Distribution')

# Histogram of sentiment scores
axes[1].hist(comments['sentiment_compound'], bins=50, color='steelblue', alpha=0.7, edgecolor='black')
axes[1].axvline(x=0, color='red', linestyle='--', label='Neutral')
axes[1].axvline(x=comments['sentiment_compound'].mean(), color='green', linestyle='--', label=f'Mean: {comments["sentiment_compound"].mean():.3f}')
axes[1].set_xlabel('Sentiment Score (-1 to +1)')
axes[1].set_ylabel('Number of Comments')
axes[1].set_title('📊 Sentiment Score Distribution')
axes[1].legend()

# Summary stats
stats_text = f"""
📊 Sentiment Summary
─────────────────────
Total Comments: {sentiment_dist['total_comments']:,}

Positive: {sentiment_dist['positive_count']:,} ({sentiment_dist['positive_pct']:.1f}%)
Neutral: {sentiment_dist['neutral_count']:,} ({sentiment_dist['neutral_pct']:.1f}%)
Negative: {sentiment_dist['negative_count']:,} ({sentiment_dist['negative_pct']:.1f}%)

Average Score: {sentiment_dist['avg_sentiment']:.3f}
Std Dev: {sentiment_dist['std_sentiment']:.3f}
"""
axes[2].text(0.1, 0.5, stats_text, transform=axes[2].transAxes, fontsize=12, 
             verticalalignment='center', fontfamily='monospace',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
axes[2].axis('off')
axes[2].set_title('📋 Summary Statistics')

plt.tight_layout()
plt.show()

## 2.2 Top Fan Favorite Comments
*Comments with the highest like counts (community consensus)*

In [ ]:
# Top Fan Favorite Comments
top_comments = analytics.top_fan_favorite_comments(10)

print("🏆 TOP 10 FAN FAVORITE COMMENTS")
print("=" * 80)

for i, row in top_comments.iterrows():
    print(f"\n#{top_comments.index.get_loc(i)+1} | 👍 {row['like_count']:,} likes")
    print(f"Video: {row.get('video_title', 'N/A')[:60]}...")
    print(f"Comment: {row['text_original'][:200]}...")
    print("-" * 80)

## 2.3 Controversy Index
*Videos with high comment-to-like ratios (heated debates)*

In [ ]:
# Controversy Index Ranking
controversial = analytics.controversy_index_ranking(15)

fig, ax = plt.subplots(figsize=(12, 6))

if not controversial.empty:
    ax.barh(range(len(controversial)), controversial['controversy_index'], color='crimson', alpha=0.7)
    ax.set_yticks(range(len(controversial)))
    ax.set_yticklabels([t[:40] + '...' if len(t) > 40 else t for t in controversial['title']])
    ax.invert_yaxis()
    ax.set_xlabel('Controversy Index (Comments / Likes)')
    ax.set_title('🔥 Most Controversial Videos (High Comment-to-Like Ratio)')
else:
    ax.text(0.5, 0.5, 'No data available', ha='center', va='center', transform=ax.transAxes)

plt.tight_layout()
plt.show()

controversial[['title', 'view_count', 'comment_count', 'like_count', 'controversy_index']].head(10)

## 2.4 Polarity vs Performance Correlation
*Do positive videos get more views?*

In [ ]:
# Calculate correlations
correlations = analytics.polarity_vs_performance_correlation()

# Calculate average sentiment per video for scatter plot
video_sentiment = comments.groupby('video_id')['sentiment_compound'].mean().reset_index()
video_sentiment.columns = ['video_id', 'avg_sentiment']
merged = videos.merge(video_sentiment, on='video_id', how='inner')

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot: Sentiment vs Views
axes[0].scatter(merged['avg_sentiment'], merged['view_count'] / 1e6, alpha=0.5, c='steelblue')
axes[0].set_xlabel('Average Comment Sentiment')
axes[0].set_ylabel('View Count (Millions)')
axes[0].set_title('📈 Sentiment vs Views')
if 'sentiment_vs_view_count' in correlations:
    axes[0].annotate(f'r = {correlations["sentiment_vs_view_count"]:.3f}', xy=(0.05, 0.95), 
                     xycoords='axes fraction', fontsize=12, 
                     bbox=dict(boxstyle='round', facecolor='wheat'))

# Correlation heatmap
corr_df = pd.DataFrame({
    'Metric': ['View Count', 'Like Count', 'Comment Count', 'Engagement Rate'],
    'Correlation': [
        correlations.get('sentiment_vs_view_count', 0),
        correlations.get('sentiment_vs_like_count', 0),
        correlations.get('sentiment_vs_comment_count', 0),
        correlations.get('sentiment_vs_engagement_rate', 0)
    ]
})
colors = ['green' if x > 0 else 'red' for x in corr_df['Correlation']]
axes[1].barh(corr_df['Metric'], corr_df['Correlation'], color=colors, alpha=0.7)
axes[1].axvline(x=0, color='black', linestyle='-')
axes[1].set_xlabel('Correlation Coefficient')
axes[1].set_title('🔗 Sentiment Correlation with Performance Metrics')

plt.tight_layout()
plt.show()

print("📊 Correlation Results:")
for key, value in correlations.items():
    print(f"  {key}: {value:.4f}")

---
# 📺 3. VIDEO PERFORMANCE METRICS

## 3.1 Engagement Rate Ranking

In [ ]:
# Video Performance Summary
performance = analytics.video_performance_summary()

print("📊 VIDEO PERFORMANCE SUMMARY")
print("=" * 50)
print(f"Total Videos: {performance['total_videos']}")
print(f"Total Views: {performance['total_views']:,}")
print(f"Total Likes: {performance['total_likes']:,}")
print(f"Total Comments: {performance['total_comments']:,}")
print(f"\nAverages per Video:")
print(f"  Views: {performance['avg_views']:,.0f}")
print(f"  Likes: {performance['avg_likes']:,.0f}")
print(f"  Comments: {performance['avg_comments']:,.0f}")
print(f"  Engagement Rate: {performance['avg_engagement_rate']:.4f}%")

In [ ]:
# Top videos by different metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Top by Engagement Rate
top_engagement = analytics.engagement_rate_ranking(10)
axes[0].barh(range(len(top_engagement)), top_engagement['engagement_rate'], color='green', alpha=0.7)
axes[0].set_yticks(range(len(top_engagement)))
axes[0].set_yticklabels([t[:30]+'...' if len(t)>30 else t for t in top_engagement['title']])
axes[0].invert_yaxis()
axes[0].set_xlabel('Engagement Rate (%)')
axes[0].set_title('🎯 Top 10 by Engagement Rate')

# Top by Views (Reach)
top_reach = analytics.video_reach_ranking(10)
axes[1].barh(range(len(top_reach)), top_reach['view_count'] / 1e6, color='steelblue', alpha=0.7)
axes[1].set_yticks(range(len(top_reach)))
axes[1].set_yticklabels([t[:30]+'...' if len(t)>30 else t for t in top_reach['title']])
axes[1].invert_yaxis()
axes[1].set_xlabel('View Count (Millions)')
axes[1].set_title('👀 Top 10 by Video Reach')

# Top by Comments (Virality)
top_viral = analytics.virality_potential_ranking(10)
axes[2].barh(range(len(top_viral)), top_viral['comment_count'], color='orange', alpha=0.7)
axes[2].set_yticks(range(len(top_viral)))
axes[2].set_yticklabels([t[:30]+'...' if len(t)>30 else t for t in top_viral['title']])
axes[2].invert_yaxis()
axes[2].set_xlabel('Comment Count')
axes[2].set_title('🚀 Top 10 by Virality (Comments)')

plt.tight_layout()
plt.show()

---
# 📝 4. CONTENT & TOPIC ANALYSIS

## 4.1 Dominant Keyword Cloud
*What are fans talking about?*

In [ ]:
# Extract keywords and create word cloud
keywords = analytics.dominant_keywords('comments', 100)

# Create word cloud
word_freq = dict(keywords)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Word Cloud
if word_freq:
    wordcloud = WordCloud(
        width=800, height=400,
        background_color='white',
        colormap='RdYlBu',
        max_words=100
    ).generate_from_frequencies(word_freq)
    
    axes[0].imshow(wordcloud, interpolation='bilinear')
    axes[0].axis('off')
    axes[0].set_title('☁️ Dominant Keywords in Comments')
else:
    axes[0].text(0.5, 0.5, 'No keywords found', ha='center', va='center', transform=axes[0].transAxes)

# Top 20 keywords bar chart
top_keywords = keywords[:20]
if top_keywords:
    words, counts = zip(*top_keywords)
    axes[1].barh(range(len(words)), counts, color='teal', alpha=0.7)
    axes[1].set_yticks(range(len(words)))
    axes[1].set_yticklabels(words)
    axes[1].invert_yaxis()
    axes[1].set_xlabel('Frequency')
    axes[1].set_title('📊 Top 20 Keywords')

plt.tight_layout()
plt.show()

## 4.2 Team Mention Frequency
*Which teams dominate the conversation?*

In [ ]:
# Team Mention Frequency
team_mentions = analytics.team_mention_frequency()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

if not team_mentions.empty:
    # Bar chart with team colors
    colors = [config.TEAM_COLORS.get(t, '#808080') for t in team_mentions['team']]
    team_names = [config.TEAMS.get(t, {}).get('full_name', t) for t in team_mentions['team']]
    
    axes[0].barh(range(len(team_mentions)), team_mentions['mention_count'], color=colors)
    axes[0].set_yticks(range(len(team_mentions)))
    axes[0].set_yticklabels(team_names)
    axes[0].invert_yaxis()
    axes[0].set_xlabel('Number of Mentions')
    axes[0].set_title('🏁 Team Mention Frequency')
    
    # Pie chart
    axes[1].pie(team_mentions['mention_count'], labels=[t[:10] for t in team_names], 
                colors=colors, autopct='%1.1f%%', startangle=90)
    axes[1].set_title('📊 Team Share of Discussion')
else:
    axes[0].text(0.5, 0.5, 'No team mentions found', ha='center', va='center', transform=axes[0].transAxes)
    axes[1].text(0.5, 0.5, 'No team mentions found', ha='center', va='center', transform=axes[1].transAxes)

plt.tight_layout()
plt.show()

team_mentions

## 4.3 Temporal Activity Heatmap
*When do uploads happen and when do fans comment?*

In [ ]:
# Temporal Activity Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Video uploads by day of week
video_temporal = analytics.temporal_activity_videos()
if not video_temporal.empty:
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    video_temporal['day_name'] = pd.Categorical(video_temporal['day_name'], categories=day_order, ordered=True)
    video_temporal = video_temporal.sort_values('day_name')
    
    axes[0, 0].bar(video_temporal['day_name'], video_temporal['video_count'], color='steelblue')
    axes[0, 0].set_xlabel('Day of Week')
    axes[0, 0].set_ylabel('Number of Videos')
    axes[0, 0].set_title('📅 Video Uploads by Day of Week')
    axes[0, 0].tick_params(axis='x', rotation=45)

# Comment activity by day
comment_temporal = analytics.temporal_activity_comments()
if not comment_temporal.empty:
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    comment_temporal['day_name'] = pd.Categorical(comment_temporal['day_name'], categories=day_order, ordered=True)
    comment_temporal = comment_temporal.sort_values('day_name')
    
    axes[0, 1].bar(comment_temporal['day_name'], comment_temporal['comment_count'], color='coral')
    axes[0, 1].set_xlabel('Day of Week')
    axes[0, 1].set_ylabel('Number of Comments')
    axes[0, 1].set_title('💬 Comment Activity by Day of Week')
    axes[0, 1].tick_params(axis='x', rotation=45)

# Heatmap of comment activity (day x hour)
heatmap_data = analytics.temporal_heatmap_data()
if not heatmap_data.empty:
    sns.heatmap(heatmap_data, cmap='YlOrRd', ax=axes[1, 0], cbar_kws={'label': 'Comment Count'})
    axes[1, 0].set_xlabel('Hour of Day (UTC)')
    axes[1, 0].set_ylabel('Day of Week')
    axes[1, 0].set_title('🔥 Comment Activity Heatmap (Day x Hour)')

# Monthly trend
if 'month' in comments.columns:
    monthly = comments.groupby('month').size()
    axes[1, 1].plot(monthly.index, monthly.values, marker='o', color='purple', linewidth=2)
    axes[1, 1].fill_between(monthly.index, monthly.values, alpha=0.3, color='purple')
    axes[1, 1].set_xlabel('Month')
    axes[1, 1].set_ylabel('Number of Comments')
    axes[1, 1].set_title('📈 Monthly Comment Trend')
    axes[1, 1].set_xticks(range(1, 13))

plt.tight_layout()
plt.show()

---
# 📋 5. SUMMARY & EXPORT

## Generate Full Report

In [ ]:
# Save all analytics to CSV files
analytics.save_report_to_csv()

print("\n✅ All analytics saved to processed data directory!")

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════╗
║           F1 YouTube Descriptive Analytics Complete!          ║
╠══════════════════════════════════════════════════════════════╣
║                                                              ║
║  📊 METRICS COMPUTED:                                        ║
║                                                              ║
║  1. DRIVER ENGAGEMENT                                        ║
║     ✓ Driver Share of Voice (SoV)                           ║
║     ✓ Driver Sentiment Score                                ║
║     ✓ Headline Impact                                       ║
║     ✓ Rivalry Intensity                                     ║
║                                                              ║
║  2. AUDIENCE SENTIMENT                                       ║
║     ✓ Global Sentiment Distribution                         ║
║     ✓ Top Fan Favorite Comments                             ║
║     ✓ Controversy Index                                     ║
║     ✓ Polarity vs Performance Correlation                   ║
║                                                              ║
║  3. VIDEO PERFORMANCE                                        ║
║     ✓ Engagement Rate Rankings                              ║
║     ✓ Video Reach Rankings                                  ║
║     ✓ Virality Potential Rankings                           ║
║                                                              ║
║  4. CONTENT ANALYSIS                                         ║
║     ✓ Dominant Keyword Cloud                                ║
║     ✓ Team Mention Frequency                                ║
║     ✓ Temporal Activity Heatmap                             ║
║                                                              ║
╚══════════════════════════════════════════════════════════════╝
""")